In [1]:
import pandas as pd

# Load dataset
# Specify the path to your CSV file
csv_file_path = '/content/AEP_hourly.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)
df = df.sort_index()
df.index.freq = pd.infer_freq(df.index)  # Infer the frequency (e.g., 'H' for hourly)


In [2]:
train_size = int(len(df) * 0.8)
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]


In [3]:
from statsmodels.tsa.arima.model import ARIMA
import itertools

# Define the range of parameters
p_values = range(0, 5)  # AR
d_values = range(0, 5)  # I
q_values = range(0, 5)  # MA

best_score, best_cfg = float('inf'), None

# Iterate over all possible combinations of p, d, q
for p, d, q in itertools.product(p_values, d_values, q_values):
    order = (p, d, q)
    try:
        model = ARIMA(train_data, order=order)
        model_fit = model.fit()
        aic = model_fit.aic
        if aic < best_score:
            best_score, best_cfg = aic, order
    except:
        continue

print('Best ARIMA Order:', best_cfg)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Best ARIMA Order: (4, 4, 3)


In [4]:
# Fit the final model with the best configuration
best_model = ARIMA(train_data, order=best_cfg)
best_model_fit = best_model.fit()

# Forecast
forecast_values = best_model_fit.forecast(steps=len(test_data))

# Calculate RMSE
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(test_data, forecast_values))
print('Test RMSE:', rmse)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/p

Test RMSE: 14917.528125478371
